In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
train = pd.read_csv("/kaggle/input/titanic/train.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_arr=[train]
test_arr=[test]

In [5]:
Age_mean = train['Age'].mean()
Age_mean_test = test['Age'].mean()

for dataset in train_arr:
    dataset['Age'] = dataset['Age'].fillna(Age_mean)
    dataset['Age'] = dataset['Age'].astype(int)
for dataset1 in test_arr:
    dataset1['Age'] = dataset1['Age'].fillna(Age_mean_test)
    dataset1['Age'] = dataset1['Age'].astype(int)

In [6]:
Fare_mean = train['Fare'].mean()
Fare_mean_test = test['Fare'].mean()
for dataset in train_arr:
    dataset['Fare'] = dataset['Fare'].fillna(Fare_mean)
    dataset['Fare'] = dataset['Fare'].astype(int)
for dataset1 in test_arr:
    dataset1['Fare'] = dataset1['Fare'].fillna(Fare_mean_test)
    dataset1['Fare'] = dataset1['Fare'].astype(int)

In [7]:
for dataset in train_arr:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
for dataset in test_arr:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [8]:
train['Sex'] = train['Sex'].map({"male": -0.5, "female": 0.5})


test['Sex'] = test['Sex'].map({"male": -0.5, "female": 0.5})

In [9]:
train['Embarked'] = train['Embarked'].map({"S": -1, "C": 0, "Q": 1})


test['Embarked'] = test['Embarked'].map({"S": -1, "C": 0, "Q": 1})

In [10]:
for dataset in train_arr:
     dataset['Embarked'] = dataset['Embarked'].astype(int)
for dataset in test_arr:
     dataset['Embarked'] = dataset['Embarked'].astype(int)

In [11]:
train = train.drop(['PassengerId'], axis=1)
passid = test['PassengerId']
test = test.drop(['PassengerId'], axis=1)

In [12]:
train = train.drop(['Ticket'], axis=1)
test = test.drop(['Ticket'], axis=1)


In [13]:
train = train.drop(['Cabin'], axis=1)
test = test.drop(['Cabin'], axis=1)

In [14]:
train = train.drop(['Name'], axis=1)
test = test.drop(['Name'], axis=1)

In [15]:
X = train.drop('Survived', axis=1)
y = train['Survived']

In [16]:
yt = np.zeros((X.shape[0],1))
for i in range(y.shape[0]):
    yt[i]= y[i]

In [17]:
X=X.T
yt=yt.T
test=test.T

In [18]:
def sigmoid(Z):
    A=1/(1+np.exp(-Z))
    return A

In [19]:
def hypo(x,w,b):
    h=np.dot(w.T,x)+b
    h=sigmoid(h)
    return h

In [20]:
def grad(x,y,w,b):
    m=x.shape[1]
    A=hypo(x,w,b)
    dw = np.dot(x,(A-y).T)/m
    db=np.sum(A-y,axis=1,keepdims=True)
    return dw,db

In [21]:
w=np.random.rand(X.shape[0],1)
b=0.5
for i in range(100000):
    dw,db = grad(X,yt,w,b)
    w = w- dw*0.001
    b = b- db*0.001

In [22]:
hp_test = hypo(test,w,b)

In [23]:
y_p_t = np.ceil(hp_test-0.5)
y_p_t=y_p_t.astype(int)

In [24]:
y_p_t.shape

(1, 418)

In [25]:
y_p_t=y_p_t.T

In [26]:
y_p_t=y_p_t.reshape((418,))

In [27]:
y_p_t.shape

(418,)

In [28]:
output = pd.DataFrame({'PassengerId': passid, 'Survived': y_p_t})
output.to_csv('my_submission.csv', index=False)